In [1]:
import pandas as pd
import datetime
from sklearn.metrics import confusion_matrix

In [2]:
# get new
live_songs = pd.read_csv('atu_database/live_songs.csv')
songs = pd.read_csv('atu_database/songs.csv')
shows = pd.read_csv('atu_database/shows.csv')
new = live_songs.merge(shows, on='show_id', how='left').drop_duplicates()
new = new.merge(songs, on='song_id', how='left').drop_duplicates()
new = new[['original_artist', 'name', 'showdate', 'footnote']]
new['date'] = pd.to_datetime(new['showdate'], format="%Y-%m-%d")
new = new.rename(columns={'original_artist':'artist'})

In [3]:
# get old
old_live_songs = pd.read_pickle('v1.0.0/atu_pull_2020-12-21/live_songs_2020-12-21.pickle')
old_songs = pd.read_pickle('v1.0.0/atu_pull_2020-12-21/songs_2020-12-21.pickle')
old_shows = pd.read_pickle('v1.0.0/atu_pull_2020-12-21/shows_2020-12-21.pickle')
old_shows['date'] = pd.to_datetime(old_shows['date'], format="%m-%d-%Y")
old = old_live_songs.merge(old_songs
                           .reset_index()
                           .rename(columns={'primary_key':'song'}),
                           on='song',
                           how='inner')
old = old.merge(old_shows
                .reset_index()
                .rename(columns={'primary_key':'show'}),
                on='show',
                how='inner')
old = old[old['parent'] == True]
old = old[['name', 'artist', 'stewart', 'stewart_with_lyrics', 'hof',  'date']]
old['date'] = pd.to_datetime(old['date'], format="%m-%d-%Y")

In [4]:
# jimmy stewart analysis
tmp = old.merge(new, on=['artist', 'date', 'name'], how='inner')
tmp['in_note'] = tmp['footnote'].astype(str).apply(lambda x: 'stewart' in x.lower())
confusion_matrix(tmp['stewart'], tmp['in_note'])

array([[35355,     6],
       [ 1329,   157]])

In [5]:
hof = pd.read_csv('data/hall_of_fame.csv', index_col=0)
hof['showdate'] = pd.to_datetime(hof['showdate'], format="%Y-%m-%d")
live_songs = pd.read_csv('atu_database/live_songs.csv')
songs = pd.read_csv('atu_database/songs.csv')
shows = pd.read_csv('atu_database/shows.csv')
new = live_songs.merge(shows, on='show_id', how='left')
new = new.merge(songs, on='song_id', how='left')
new['showdate'] = pd.to_datetime(new['showdate'], format="%Y-%m-%d")

In [6]:
hof.merge(new, on=['showdate', 'name'], how='left').drop_duplicates()

,name,showdate,live_song_id,show_id,song_id,setnumber,position,transition,footnote,isjamchart,...,artist,venue_id,tourname,showorder,shownotes,opener,soundcheck,slug,original_artist,isoriginal
0,Attachments,2019-06-22,251376,1561302540,920,2,11,>,NaN,1,...,Umphrey's McGee,red-rocks-amphitheatre-morrison-co-usa,No Tour Name,2.0,last With a Little Help from My Friends 2008-1...,Turkuaz,NaN,attachments,Umphrey's McGee,True
1,Eat,2019-08-30,253369,1567546902,33,1,5,->,NaN,0,...,Umphrey's McGee,stone-pony-outdoors-asbury-park-nj-usa,No Tour Name,1.0,last Muff II: The Revenge 2017-01-29 (241 shows),Big Something,NaN,eat,Umphrey's McGee,True
2,JaJunk,2019-04-03,249518,1554384690,54,2,9,>,&quot;Jimmy Stewart&quot; with lyrics,0,...,Umphrey's McGee,belly-up-aspen-aspen-co-usa,No Tour Name,1.0,last London Calling 2011-01-31 (833 shows),NaN,NaN,jajunk,Umphrey's McGee,True
3,JaJunk,2019-04-03,249520,1554384690,54,2,11,",",NaN,0,...,Umphrey's McGee,belly-up-aspen-aspen-co-usa,No Tour Name,1.0,last London Calling 2011-01-31 (833 shows),NaN,NaN,jajunk,Umphrey's McGee,True
4,Puppet String,2017-02-18,217809,1487521457,99,1,6,",",with Jake on keys,0,...,Umphrey's McGee,exploreashevillecom-arena-asheville-nc-usa,No Tour Name,1.0,NaN,Greensky Bluegrass,NaN,puppet-string,Umphrey's McGee,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Utopian Fir,2010-02-11,21463,1335320633,140,2,13,>,NaN,0,...,Umphrey's McGee,the-orange-peel-asheville-nc-usa,No Tour Name,1.0,NaN,NaN,NaN,utopian-fir,Umphrey's McGee,True
213,Utopian Fir,2010-02-11,21465,1335320633,140,2,15,>,NaN,0,...,Umphrey's McGee,the-orange-peel-asheville-nc-usa,No Tour Name,1.0,NaN,NaN,NaN,utopian-fir,Umphrey's McGee,True
214,Conduit,2013-08-17,145464,1376792880,24,1,8,",",NaN,1,...,Umphrey's McGee,firstmerit-bank-pavilion-at-northerly-island-c...,No Tour Name,1.0,played after STS9\r\nTribute to the Spinal Sha...,NaN,NaN,conduit,Umphrey's McGee,True
215,Utopian Fir,2016-03-10,200197,1457672452,140,1,7,->,with The Ocean (Led Zeppelin) tease,0,...,Umphrey's McGee,the-hive-sandpoint-id-usa,No Tour Name,1.0,NaN,NaN,NaN,utopian-fir,Umphrey's McGee,True


In [7]:
# notes
# Bright Lights, Big City (website says Karl Engelmann. data says Umphreys)
# Ringo (website says Ali Baba's Tahini. data says Umphreys)
# ...
# Multiple Ali Baba's Tahini and memebers (Karl Engelmann) are now Umphreys?